In [2]:
import pandas as pd 
from pandasql import sqldf


In [3]:
appearances = pd.read_csv("appearances.csv")
player_position = pd.read_csv("player_position.csv")
teamstats = pd.read_csv("teamstats.csv")
teams = pd.read_csv("teams.csv")
players = pd.read_csv("players.csv", encoding = "ISO-8859-1")
shots = pd.read_csv("shots.csv")
leagues = pd.read_csv("leagues.csv")
games = pd.read_csv("games.csv")

In [4]:
games_teams = pd.read_csv("games_teams.csv")

In [63]:
teams_leagues = sqldf("select teamID, name, leagueID, country from games_teams group by teamID, name, leagueID, country")

In [17]:
test = sqldf("""
            select a.*, b.name as team_name
            from teamstats a
            inner join teams b 
            on a.teamID = b.teamID
            """)

In [42]:
test

,gameID,teamID,season,date,location,goals,xGoals,shots,shotsOnTarget,deep,ppda,fouls,corners,yellowCards,redCards,result,team_name
0,81,89,2015,2015-08-08 15:45:00,h,1,0.627539,9,1,4,13.8261,12,1,2.0,0,W,Manchester United
1,81,82,2015,2015-08-08 15:45:00,a,0,0.674600,9,4,10,8.2188,12,2,3.0,0,L,Tottenham
2,82,73,2015,2015-08-08 18:00:00,h,0,0.876106,11,2,11,6.9000,13,6,3.0,0,L,Bournemouth
3,82,71,2015,2015-08-08 18:00:00,a,1,0.782253,7,3,2,11.8462,13,3,4.0,0,W,Aston Villa
4,83,72,2015,2015-08-08 18:00:00,h,2,0.604226,10,5,5,6.6500,7,8,1.0,0,D,Everton
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25355,16133,235,2020,2021-05-23 19:00:00,a,0,0.357583,9,2,0,39.7273,10,3,0.0,0,L,Nimes
25356,16134,175,2020,2021-05-23 19:00:00,h,0,1.460500,19,5,6,7.5600,13,9,1.0,0,L,Saint-Etienne
25357,16134,181,2020,2021-05-23 19:00:00,a,1,1.380290,10,2,3,14.7200,10,3,0.0,0,W,Dijon
25358,16135,225,2020,2021-05-23 19:00:00,h,1,0.323960,6,2,1,15.1000,17,2,1.0,0,D,Strasbourg


In [36]:
test2 = sqldf("""
              select team_name, season, result, sum(goals) as goals, count(1) as games
              from test
              group by 1,2,3
              """)

In [37]:
test3 = sqldf("select *, case when result = 'D' then 1 when result = 'W' then 3 else 0 end as points from test2 ")

In [66]:
test4 = sqldf("select team_name, season, sum(goals) as total_goals, sum(games) as total_games, sum(points * games) as total_points from test3 group by team_name, season")

In [68]:
test5 =sqldf("""select *
         from test4 a 
         inner join teams_leagues b 
         on a.team_name = b.name
      """)

In [69]:
test5

,team_name,season,total_goals,total_games,total_points,teamID,name,leagueID,country
0,AC Milan,2014,56,38,52,111,AC Milan,Serie A,Italy
1,AC Milan,2015,49,38,57,111,AC Milan,Serie A,Italy
2,AC Milan,2016,57,38,63,111,AC Milan,Serie A,Italy
3,AC Milan,2017,56,38,64,111,AC Milan,Serie A,Italy
4,AC Milan,2018,55,38,68,111,AC Milan,Serie A,Italy
...,...,...,...,...,...,...,...,...,...
681,Wolfsburg,2019,48,34,49,131,Wolfsburg,Bundesliga,Germany
682,Wolfsburg,2020,61,34,61,131,Wolfsburg,Bundesliga,Germany
683,Wolverhampton Wanderers,2018,47,38,57,229,Wolverhampton Wanderers,Premier League,United Kingdom
684,Wolverhampton Wanderers,2019,51,38,59,229,Wolverhampton Wanderers,Premier League,United Kingdom


In [82]:
standings = sqldf("""select *,   rank() over (partition by leagueID, season
                                 order by total_points desc) as standing
        from test5
        """)

In [112]:
standings

,team_name,season,total_goals,total_games,total_points,teamID,name,leagueID,country,standing
0,Bayern Munich,2014,80,34,79,117,Bayern Munich,Bundesliga,Germany,1
1,Wolfsburg,2014,72,34,69,131,Wolfsburg,Bundesliga,Germany,2
2,Borussia M.Gladbach,2014,53,34,66,130,Borussia M.Gladbach,Bundesliga,Germany,3
3,Bayer Leverkusen,2014,62,34,61,119,Bayer Leverkusen,Bundesliga,Germany,4
4,Augsburg,2014,43,34,49,121,Augsburg,Bundesliga,Germany,5
...,...,...,...,...,...,...,...,...,...,...
681,Cagliari,2020,43,38,37,116,Cagliari,Serie A,Italy,16
682,Torino,2020,50,38,37,113,Torino,Serie A,Italy,16
683,Benevento,2020,40,38,33,222,Benevento,Serie A,Italy,18
684,Crotone,2020,45,38,23,115,Crotone,Serie A,Italy,19


In [111]:
sqldf("select * from standings a inner join player_position

,playerID,name,position,leagueID
0,560,Sergio Romero,GK,Serie A
1,557,Matteo Darmian,DR,Serie A
2,548,Daley Blind,DC,Premier League
3,628,Chris Smalling,DC,Premier League
4,1006,Luke Shaw,DL,Premier League
...,...,...,...,...
6710,7396,Loic Bessile,DC,Ligue 1
6711,9566,Yanis Lhéry,Sub,Ligue 1
6712,9565,Mathys Saban,Sub,Ligue 1
6713,9568,Charles Costes,Sub,Ligue 1


In [110]:
player_position

,playerID,name,position,leagueID
0,560,Sergio Romero,GK,Serie A
1,557,Matteo Darmian,DR,Serie A
2,548,Daley Blind,DC,Premier League
3,628,Chris Smalling,DC,Premier League
4,1006,Luke Shaw,DL,Premier League
...,...,...,...,...
6710,7396,Loic Bessile,DC,Ligue 1
6711,9566,Yanis Lhéry,Sub,Ligue 1
6712,9565,Mathys Saban,Sub,Ligue 1
6713,9568,Charles Costes,Sub,Ligue 1


In [134]:
sqldf("select * from players where name = 'Lionel Messi' or name like '%Ronaldo%'")

,playerID,name
0,2028,Ronaldo
1,2097,Lionel Messi
2,2371,Cristiano Ronaldo
3,7097,Ronaldo Vieira


In [135]:
sqldf("select playerID, sum(goals) from appearances where playerID in (2371,2097) group by 1")

,playerID,sum(goals)
0,2097,231
1,2371,215


In [177]:
page2 = sqldf("""select a.*, b.season, b.date
                 from ronaldo_messi_appearances a 
                 inner join games b 
                 on a.gameID=b.gameID""")

In [179]:
page2

,gameID,playerID,player_name,team,goals,shots,xGoals,xGoalsChain,xGoalsBuildup,assists,keyPasses,xAssists,leagueID,season,date
0,1404,2097,Messi,Barcelona,0,4,0.906503,0.277046,0.048860,0,1,0.134688,4,2015,2015-08-23 20:30:00
1,1405,2371,Ronaldo,Real Madrid,0,10,0.563143,0.501273,0.122807,0,0,0.000000,4,2015,2015-08-23 22:30:00
2,1411,2097,Messi,Barcelona,0,7,0.409798,0.607837,0.224740,0,2,0.475536,4,2015,2015-08-29 22:30:00
3,1413,2371,Ronaldo,Real Madrid,0,6,1.421785,1.861596,0.630725,0,1,0.026232,4,2015,2015-08-30 00:30:00
4,1420,2371,Ronaldo,Real Madrid,5,7,3.226308,3.414285,0.000000,1,3,0.931255,4,2015,2015-09-12 18:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
462,15748,2371,Ronaldo,Juventus,0,5,0.606366,0.226862,0.035610,0,1,0.058229,2,2020,2021-04-25 13:00:00
463,15762,2371,Ronaldo,Juventus,2,6,2.053876,0.673986,0.065147,0,1,0.065147,2,2020,2021-05-02 16:00:00
464,15771,2371,Ronaldo,Juventus,0,2,0.073078,0.176236,0.052933,0,1,0.066498,2,2020,2021-05-09 18:45:00
465,15779,2371,Ronaldo,Juventus,1,4,0.473355,1.169019,0.591276,0,2,0.104389,2,2020,2021-05-12 18:45:00


In [186]:
page2_data = sqldf("""select playerID,player_name, season, sum(goals) as goals, sum(assists) as assists, sum(shots) as shots, sum(goals)+sum(assists) as `g+a`
            from page2
            group by 1,2,3 """)

In [187]:
page2_data.to_csv('page2_data.csv',index=False)

In [164]:
page2

,gameID,playerID,goals,shots,xGoals,xGoalsChain,xGoalsBuildup,assists,keyPasses,xAssists,leagueID,season,date
0,1404,2097,0,4,0.906503,0.277046,0.048860,0,1,0.134688,4,2015,2015-08-23 20:30:00
1,1405,2371,0,10,0.563143,0.501273,0.122807,0,0,0.000000,4,2015,2015-08-23 22:30:00
2,1411,2097,0,7,0.409798,0.607837,0.224740,0,2,0.475536,4,2015,2015-08-29 22:30:00
3,1413,2371,0,6,1.421785,1.861596,0.630725,0,1,0.026232,4,2015,2015-08-30 00:30:00
4,1420,2371,5,7,3.226308,3.414285,0.000000,1,3,0.931255,4,2015,2015-09-12 18:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
462,15748,2371,0,5,0.606366,0.226862,0.035610,0,1,0.058229,2,2020,2021-04-25 13:00:00
463,15762,2371,2,6,2.053876,0.673986,0.065147,0,1,0.065147,2,2020,2021-05-02 16:00:00
464,15771,2371,0,2,0.073078,0.176236,0.052933,0,1,0.066498,2,2020,2021-05-09 18:45:00
465,15779,2371,1,4,0.473355,1.169019,0.591276,0,2,0.104389,2,2020,2021-05-12 18:45:00


In [19]:
shots

,gameID,shooterID,assisterID,minute,situation,lastAction,shotType,shotResult,xGoal,positionX,positionY
0,81,554,NaN,27,DirectFreekick,Standard,LeftFoot,BlockedShot,0.104347,0.794,0.421
1,81,555,631.0,27,SetPiece,Pass,RightFoot,BlockedShot,0.064342,0.860,0.627
2,81,554,629.0,35,OpenPlay,Pass,LeftFoot,BlockedShot,0.057157,0.843,0.333
3,81,554,NaN,35,OpenPlay,Tackle,LeftFoot,MissedShots,0.092141,0.848,0.533
4,81,555,654.0,40,OpenPlay,BallRecovery,RightFoot,BlockedShot,0.035742,0.812,0.707
...,...,...,...,...,...,...,...,...,...,...,...
324538,16135,6615,8651.0,19,SetPiece,Aerial,Head,MissedShots,0.039172,0.899,0.511
324539,16135,6615,8651.0,54,SetPiece,Cross,LeftFoot,Goal,0.372809,0.902,0.482
324540,16135,3464,NaN,70,OpenPlay,NaN,LeftFoot,MissedShots,0.009242,0.738,0.774
324541,16135,8651,4882.0,72,OpenPlay,Cross,Head,BlockedShot,0.045374,0.882,0.543


In [6]:
ronaldo_messi_appearances = sqldf("select *, case when playerID = 2097 then 'Messi' else 'Ronaldo' end as player_name, case when playerID = 2371 and leagueID = 4 then 'Real Madrid' when playerID = 2371 and leagueID != 4 then 'Juventus' else 'Barcelona' end as team from appearances where playerID in (2371,2097)")[['gameID', 'playerID','player_name', 'team', 'goals', 'shots', 'xGoals', 'xGoalsChain', 'xGoalsBuildup', 'assists', 'keyPasses', 'xAssists', 'leagueID']]

In [7]:
ronaldo_messi_appearances

,gameID,playerID,player_name,team,goals,shots,xGoals,xGoalsChain,xGoalsBuildup,assists,keyPasses,xAssists,leagueID
0,1404,2097,Messi,Barcelona,0,4,0.906503,0.277046,0.048860,0,1,0.134688,4
1,1405,2371,Ronaldo,Real Madrid,0,10,0.563143,0.501273,0.122807,0,0,0.000000,4
2,1411,2097,Messi,Barcelona,0,7,0.409798,0.607837,0.224740,0,2,0.475536,4
3,1413,2371,Ronaldo,Real Madrid,0,6,1.421785,1.861596,0.630725,0,1,0.026232,4
4,1420,2371,Ronaldo,Real Madrid,5,7,3.226308,3.414285,0.000000,1,3,0.931255,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
462,15748,2371,Ronaldo,Juventus,0,5,0.606366,0.226862,0.035610,0,1,0.058229,2
463,15762,2371,Ronaldo,Juventus,2,6,2.053876,0.673986,0.065147,0,1,0.065147,2
464,15771,2371,Ronaldo,Juventus,0,2,0.073078,0.176236,0.052933,0,1,0.066498,2
465,15779,2371,Ronaldo,Juventus,1,4,0.473355,1.169019,0.591276,0,2,0.104389,2


In [20]:
ronaldo_messi_shots = sqldf("""select a.*, c.season, situation, shotType, shotResult, xGoal, positionX, positionY
        from ronaldo_messi_appearances a 
        inner join shots b 
        on a.playerID = b.shooterID
        and a.gameID=b.gameID
        inner join games c 
        on a.gameID = c.gameID;""")

In [21]:
ronaldo_messi_shots

,gameID,playerID,player_name,team,goals,shots,xGoals,xGoalsChain,xGoalsBuildup,assists,keyPasses,xAssists,leagueID,season,situation,shotType,shotResult,xGoal,positionX,positionY
0,1404,2097,Messi,Barcelona,0,4,0.906503,0.277046,0.048860,0,1,0.134688,4,2015,DirectFreekick,LeftFoot,BlockedShot,0.055329,0.741,0.364
1,1404,2097,Messi,Barcelona,0,4,0.906503,0.277046,0.048860,0,1,0.134688,4,2015,OpenPlay,LeftFoot,SavedShot,0.093498,0.863,0.532
2,1404,2097,Messi,Barcelona,0,4,0.906503,0.277046,0.048860,0,1,0.134688,4,2015,Penalty,LeftFoot,SavedShot,0.743278,0.885,0.500
3,1404,2097,Messi,Barcelona,0,4,0.906503,0.277046,0.048860,0,1,0.134688,4,2015,SetPiece,LeftFoot,MissedShots,0.014398,0.743,0.425
4,1405,2371,Ronaldo,Real Madrid,0,10,0.563143,0.501273,0.122807,0,0,0.000000,4,2015,DirectFreekick,RightFoot,BlockedShot,0.056615,0.765,0.699
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2850,15779,2371,Ronaldo,Juventus,1,4,0.473355,1.169019,0.591276,0,2,0.104389,2,2020,OpenPlay,RightFoot,BlockedShot,0.014912,0.747,0.766
2851,15779,2371,Ronaldo,Juventus,1,4,0.473355,1.169019,0.591276,0,2,0.104389,2,2020,OpenPlay,RightFoot,SavedShot,0.034178,0.806,0.702
2852,15779,2371,Ronaldo,Juventus,1,4,0.473355,1.169019,0.591276,0,2,0.104389,2,2020,OpenPlay,RightFoot,ShotOnPost,0.057282,0.860,0.715
2853,15790,2371,Ronaldo,Juventus,1,2,1.362895,0.098848,0.024203,0,1,0.074645,2,2020,Penalty,RightFoot,SavedShot,0.761299,0.885,0.500


In [22]:
ronaldo_messi_shots.to_csv('ronaldo_messi_shots.csv', index = False)

In [18]:
ronaldo_messi_shots

,gameID,playerID,player_name,team,goals,shots,xGoals,xGoalsChain,xGoalsBuildup,assists,keyPasses,xAssists,leagueID,season,situation,shotType,shotResult,xGoal,season
0,1404,2097,Messi,Barcelona,0,4,0.906503,0.277046,0.048860,0,1,0.134688,4,2015,DirectFreekick,LeftFoot,BlockedShot,0.055329,2015
1,1404,2097,Messi,Barcelona,0,4,0.906503,0.277046,0.048860,0,1,0.134688,4,2015,OpenPlay,LeftFoot,SavedShot,0.093498,2015
2,1404,2097,Messi,Barcelona,0,4,0.906503,0.277046,0.048860,0,1,0.134688,4,2015,Penalty,LeftFoot,SavedShot,0.743278,2015
3,1404,2097,Messi,Barcelona,0,4,0.906503,0.277046,0.048860,0,1,0.134688,4,2015,SetPiece,LeftFoot,MissedShots,0.014398,2015
4,1405,2371,Ronaldo,Real Madrid,0,10,0.563143,0.501273,0.122807,0,0,0.000000,4,2015,DirectFreekick,RightFoot,BlockedShot,0.056615,2015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2850,15779,2371,Ronaldo,Juventus,1,4,0.473355,1.169019,0.591276,0,2,0.104389,2,2020,OpenPlay,RightFoot,BlockedShot,0.014912,2020
2851,15779,2371,Ronaldo,Juventus,1,4,0.473355,1.169019,0.591276,0,2,0.104389,2,2020,OpenPlay,RightFoot,SavedShot,0.034178,2020
2852,15779,2371,Ronaldo,Juventus,1,4,0.473355,1.169019,0.591276,0,2,0.104389,2,2020,OpenPlay,RightFoot,ShotOnPost,0.057282,2020
2853,15790,2371,Ronaldo,Juventus,1,2,1.362895,0.098848,0.024203,0,1,0.074645,2,2020,Penalty,RightFoot,SavedShot,0.761299,2020
